## QCGPU

[qcgpu source](https://github.com/kwccoin/qcgpu) and

[research paper](https://arxiv.org/pdf/1805.00988.pdf)

Need to be installed and not sure it can be run under mac
and also it run in parallel it seems at least for the last 2 examples


In [20]:
# -*- coding: utf-8 -*-

"""
Bell State / EPR Pair
=====================

The Bell State, also known as the EPR pair (after Einstein, Podosky and Rosen)
is the simplest example of entanglement.

The Bell State is defined as the maximally entangled quantum state of two qubits.
"""

def bell_state():
    import qcgpu

    print("Creating Bell State")

    state = qcgpu.State(2)

    state.h(0)
    state.cx(0, 1)

    print("Measurement Results:")
    print(state.measure(samples = 1000))

if __name__== "__main__":
  bell_state()

Creating Bell State
Measurement Results:
{'00': 514, '11': 486}


In [21]:
# -*- coding: utf-8 -*-

"""
Bernstein-Vazirani Algorithm
============================

This algorithm finds a hidden integer :math:`a \in \{ 0, 1\}^n` from
an oracle :math:`f_a` which returns a bit :math:`a \cdot x \equiv \sum_i a_i x_i \mod 2`
for an input :math:`x \in \{0,1\}^n`.

A classical oracle returns :math:`f_a(x) = a \dot x \mod 2`, while the quantum oracle
must be queried with superpositions of input :math:`x`'s.

To solve this problem classically, the hidden integer can be found by checking the
oracle with the inputs :math:`x = 1,2,\dots,2^i,2^{n-1}`, where each
query reveals the :math:`i`th bit of :math:`a` (:math:`a_i`).
This is the optimal classical solution, and is :math:`O(n)`. Using a quantum oracle and the
Bernstein-Vazirani algorithm, :math:`a` can be found with just one query to the oracle.

The Algorithm
-------------

1. Initialize :math:`n` qubits in the state :math:`\lvert 0, \dots, 0\rangle`.
2. Apply the Hadamard gate :math:`H` to each qubit.
3. Apply the inner product oracle.
4. Apply the Hadamard gate :math:`H` to each qubit.
5. Measure the register

From this procedure, we find that the registers measured value is equal to that of
the original hidden integer.
"""

def bernstein_vazirani():
    import qcgpu

    num_qubits = 7 # The number of qubits to use
    a = 101 # The hidden integer, bitstring is 1100101

    register = qcgpu.State(num_qubits) # Create a new quantum register

    register.apply_all(qcgpu.gate.h()) # Apply a hadamard gate to each qubit

    # Apply the inner products oracle
    for i in range(num_qubits):
        if a & (1 << i) != 0:
            register.z(i)

    register.apply_all(qcgpu.gate.h()) # Apply a hadamard gate to each qubit

    results = register.measure(samples=1000) # Measure the register (sample 1000 times)
    print(results)

if __name__== "__main__":
  bernstein_vazirani()

{'1100101': 1000}


In [22]:
import qcgpu

# 3 qubits, f(x) = x_0 NOT x_1 x_2
# Balanced
balanced_state = qcgpu.State(3)

balanced_state.apply_all(qcgpu.gate.h())

# Oracle U_f
balanced_state.h(2)
balanced_state.z(0)
balanced_state.cx(1, 2)
balanced_state.h(2)

balanced_state.apply_all(qcgpu.gate.h())

outcomes = balanced_state.measure(samples = 1000)

if int(max(outcomes, key=outcomes.get)) == 0:
    print('constant')
else:
    print('balanced')


# 3 qubits, f(x) = 0
# Constant
constant_state = qcgpu.State(3)
print(constant_state)

constant_state.apply_all(qcgpu.gate.h())
print(constant_state)

# Oracle is equivalent to the identity gate, 
# thus has no effect on the state

constant_state.apply_all(qcgpu.gate.h())
print(constant_state)

outcomes = constant_state.measure(samples = 1000)
print(outcomes)

if int(max(outcomes, key=outcomes.get)) == 0:
    print('constant')
else:
    print('balanced')
    print(constant_state)
    print(outcomes)
    
# it prints both and hence it is both ==0 and !=0 !!!
# entangled even on classical simulator haha
# actually only balanced come out and no constant ...

balanced
[[cl.Array(1.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)]]
[[cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)
  cl.Array(0.35355335+0.j, dtype=complex64)]]
[[cl.Array(0.9999999+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64) cl.Array(0.+0.j, dtype=complex64)
  cl.Array(0.+0.j, dtype=complex64)]]
{'000': 1000}
constant


In [ ]:
"""
only this run on gpu 
and hence parallel and wait state and hold up all resources

QFT
=====================

This is an implementation of the quantum Fourier transform.
"""

import qcgpu
import math

def qft():
    print('start')
    state = qcgpu.State(24)
    print(state)
    num_qubits = state.num_qubits

    for j in range(num_qubits):
        for k in range(j):
            state.cu1(j, k, math.pi/float(2**(j-k)))
        state.h(j)
        
    print(state)

if __name__== "__main__":
    qft()